In [11]:
import numpy
import functools 

In [12]:
# hold the weights of all layer in NN  256x256 = 65,536 inputs
def layers_weights(last_layer, initial=True):
    network_weights = []
    layer = last_layer
    
    while "previous_layer" in layer.__init__.__code__.co_varnames:
        if initial == True:
            network_weights.append(layer.initial_weights)
        elif initial == False:
            network_weights.append(layer.trained_weights)
        else:
            raise ValueError("Unexpected value to the 'initial' parameter: {initial}.".format(initial=initial))
        
        layer = layer.previous_layer

    if not (type(layer) is InputLayer):
         raise TypeError("The first layer in the network architecture must be an input layer.")
    
    network_weights.reverse()
    return numpy.array(network_weights)



# Convert the layers_weights to vector 
def layers_weights_as_vector(last_layer, initial = True):
    network_weights = []
    layer = last_layer
    
    while "previous_layer" in layer.__init__.__code__.co_varnames:
        if initial == True:
            vector = numpy.reshape(layer.initial_weights, newshape=(layer.initial_weights.size))
            network_weights.extend(vector)
        elif initial == False:
            vector = numpy.reshape(layer.trained_weights, newshape=(layer.trained_weights.size))
            network_weights.extend(vector)
        else:
            raise ValueError("Unexpected value to the 'initial' parameter: {initial}.".format(initial=initial))

        layer = layer.previous_layer
        
    if not (type(layer) is InputLayer):
        raise TypeError("The first layer in the network architecture must be an input layer.")
    
    network_weights.reverse()
    return numpy.array(network_weights)



#convert vector weights to matrix
def layers_weights_as_matrix(last_layer, vector_weights):
    network_weight = []
    
    start = 0
    layer = last_layer
    vector_weights= vector_weights[::-1]
    
    while "previous_layer" in layer.__init__.__code__.co_varnames:
        layer_weights_shape = layer.initial_weights.shape
        layer_weights_size = layer.initial_weights.shape
        
        vector_weight = vector_weights[start:start + layer_weights_size]
        matrix = numpy.reshape(vector_weight, newshape=(layer_weights_shape))
        network_weights.append(matrix)
        
        start = start + layer_weights_size
        
        layer = layer.previous_layer
    
    if not (type(layer) is InputLayer):
        raise TypeError("The first layer in the network architecture must be an input layer.")
    
    network_weights.reverse()
    return network_weights
        
    

def layers_activations(last_layer):
    activations = []
    layer = last_layer
    
    while "previous_layer" in layer.__init__.__code__.co_varnames:
        activations.append(layer.activation_function)
        
        layer = layer.previous_layer
        
    if not (type(layer) is InputLayer):
        raise TypeError("The first layer in the network architecture must be an input layer.")
    
    network_weights.reverse()
    return network_weights
        

    
def sigmoid(input):
    if type(input) in [list, tuple]:
        input = numpy.array(input)
        
    return 1.0/(1 + numpy.exp(-1 * input))



# apply rectified linear unit (relu) function
def relu(input):
    if not (type(input) in [list, tuple, numpy.ndarray]):
        if input < 0:
            return 0
        else:
            return input
    elif type(input) in [list, tuple]:
        input = numpy.array(input)

    result = input
    result[input < 0] = 0

    return result
    

    
def softmax(layer_outputs):
    return layer_outputs / (numpy.sum(layer_outputs) + 0.000001)



def train(num_epochs, 
          last_layer, 
          data_inputs, 
          data_outputs,
          problem_type="classification",
          learning_rate=0.01):
    
   # fetch the initial weights of the layer
    weights = layers_weights(last_layer, initial=True)
    activations = layers_activations(last_layer)
    
    nnNetwork_error = 0
    
    
    
    
# how far the target to the predicted    
    for epoch in range(epochs):
        print("Epoch ", epoch)
        for sample_idx in range(data_inputs.shape[0]):
            r1 = data_inputs[sample_idx, :]
            for idx in range(len(weights) -1):
                curr_weights = weights[idx]
                r1 = numpy.matmul(r1, curr_weights)
                
                if activations[idx] == "relu":
                    r1 = relu(r1)
                elif activations[idx] == "sigmoid":
                    r1 = sigmoid(r1)
                elif activations[idx] == "softmax":
                    r1 = softmax(r1)
                elif activations[idx] == None:
                    pass
                
            curr_weights = weights[-1]
            r1 = numpy.matmul(r1, curr_weights)
            
            if problem_type == "classification":
                prediction = numpy.where(r1 == numpy.max(r1))[0][0]
            else:
                prediction = r1
            
            network_error = network_error + numpy.mean(numpy.abs((prediction - data_outputs[sample_idx])))

# update network weights after completing an epoch 
        weights = update_weights(weights=weights,
                                 network_error=network_error,
                                 learning_rate=learning_rate)
    
    update_layers_trained_weights(last_layer, weights)
        

        
# update weight using lrate        
def update_weights(weights, network_err, l_rate):
    for layer_idx in range(len(weights)):
        weights[layer_idx] = network_err * l_rate * weights[layer_idx]
        
    return weights




def update_layers_trained_weights(last_layer, final_weights):
    layer = last_layer
    layer_idx = len(final_weights) - 1
    
    while "previous_layer" in  layer.__init__.__code__.co_varnames:
        layer.trained_weights = final_weights[layer_idx]
        
        layer_idx = layer_idx - 1
        
        layer = layer.previous_layer
        
        
def predict(last_layer, data_inputs, problem_type="classification"):
    
    weights = layers_weights(last_layer, initial=False)
    activations = layers_activations(last_layer)

    
    if len(weights) != len(activations):
        raise TypeError("The length of layers {num_layers} is not equal to the number of activations functions {num_activations} and they must be equal.".format(num_layers=len(weights), num_activations=len(activations)))
        
    predictions = []
    for sample_idx in range(data_inputs.shape[0]):
        r1 = data_inputs[sample_idx, :]
        for curr_weights, activation in zip(weights, activations):
            r1 = numpy.matmul(r1, curr_weights)
            if activation == "relu":
                r1 = relu(r1)
            elif activation == "sigmoid":
                r1 = sigmoid(r1)
            elif activation == "softmax":
                r1 = softmax(r1)
            elif activation == None:
                pass

        if problem_type == "classification":
            prediction = numpy.where(r1 == numpy.max(r1))[0][0]
        else:
            prediction = r1

        predictions.append(prediction)

    return predictions




def to_vector(array):
    if not (type(array) is numpy.ndarray):
        raise TypeError("An input of type numpy.ndarray is expected but an input of type {in_type} found.".format(in_type=type(array)))
    return numpy.reshape(array, newshape=(array.size))





def to_array(vector, shape):
    
    if not (type(vector) is numpy.ndarray):
        raise TypeError("An input of type numpy.ndarray is expected but an input of type {in_type} found.".format(in_type=type(vector)))
        
    if vector.ndim > 1:
        raise ValueError("A 1D NumPy array is expected but an array of {ndim} dimensions found.".format(ndim=vector.ndim))
        
    if vector.size != functools.reduce(lambda x,y:x*y, shape, 1):
        raise ValueError("Mismatch between the vector length and the array shape. A vector of length {vector_length} cannot be converted into a array of shape ({array_shape}).".
                         format(vector_length=vector.size, array_shape=shape))
        
    return numpy.reshape(vector, newshape=shape)



In [13]:
class InputLayer:
    def __init__(self, num_inputs):
          if num_inputs <= 0:
            raise ValueError("Number of input neurons cannot be <= 0. Please pass a valid value to the 'num_inputs' parameter.")
        
          self.num_neurons = num_inputs
            
            
            
class DenseLayer:
    def __init__(self, num_neurons, previous_layer, activation_function="sigmoid"):
        
        if num_neurons <= 0:
            raise ValueError("Number of neurons cannot be <= 0. Please pass a valid value to the 'num_neurons' parameter.")
            
        # Number of neurons in the dense layer.
        self.num_neurons = num_neurons

        supported_activation_functions = ("sigmoid", "relu", "softmax", "None")
        if not (activation_function in supported_activation_functions):
            raise ValueError("The specified activation function '{activation_function}' is not among the supported activation functions {supported_activation_functions}. Please use one of the supported functions.".format(activation_function=activation_function, supported_activation_functions=supported_activation_functions))
        self.activation_function = activation_function

        if previous_layer is None:
            raise TypeError("The previous layer cannot be of Type 'None'. Please pass a valid layer to the 'previous_layer' parameter.")
       
        self.previous_layer = previous_layer

        # Initialize the weights of the layer.
        self.initial_weights = numpy.random.uniform(low=-0.1,
                                                    high=0.1,
                                                    size=(previous_layer.num_neurons, num_neurons))
        
        
        self.trained_weights = self.initial_weights.copy()